<a href="https://colab.research.google.com/github/ssosoo/2024_DS60/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

텐서플로를 사용해 MNIST 숫자 데이터를 분류하는 CNN을 구현한다.

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np


In [8]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

MNIST 데이터는 28x28의 픽셀 테이터로, 흑백 사진과 같이 0~255까지의 그레이스케일을 가진다.

In [9]:
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


In [10]:
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


In [12]:
print("test data has " + str(x_test.shape[0]) + ' samples')
print("every test data is " + str(x_test.shape[1])+" * "+str(x_test.shape[2])+" image")

test data has 10000 samples
every test data is 28 * 28 image



*   첫 번째 차원 (10000): 테스트 이미지의 총 개수


* 두 번째 차원 (28): 각 이미지의 높이 (픽셀)

* 세 번째 차원 (28): 각 이미지의 너비 (픽셀)

In [15]:
x_train = np.reshape(x_train, (60000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [16]:
#데이터 정규화
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_test /= gray_scale

In [19]:
#손실 함수에서 크로스 엔트로피를 계산하기 위해 실제값은 원 핫 인코딩 값으로 변경합니다.
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

원 핫 인코딩


원본 레이블: [3, 5, 1]
원 핫 인코딩 후:

Copy[[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]


 이렇게, 이진으로 변환하여 신경망의 효율을 높인다.

In [21]:
# CNN 구현

model = Sequential()
# conv1
model.add(Conv2D(16, kernel_size=(5, 5),
                 activation = 'relu',
                 input_shape = (28,28,1), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# conv2
model.add(Conv2D(32, kernel_size=(5,5), activation = 'relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
# h1: 128노드, h2: 10노드
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# 모델 최적화
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

# 두 번 이상 모델의 개선이 없을 경우 중료
callbacks = [EarlyStopping(monitor='val_accuracy', patience=2,
                           restore_best_weights=False),
             ModelCheckpoint(filepath='best_model.keras', monitor='val_accuracy',
                             save_best_only=True)]

In [23]:
model.fit(x_train, y_train,
          batch_size=500,
          epochs=5,
          verbose=1,
          validation_split=0.1, #학습 데이터의 10%를 검증 데이터로 사용
          callbacks = callbacks)

Epoch 1/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 69s 617ms/step - accuracy: 0.7236 - loss: 0.9468 - val_accuracy: 0.9700 - val_loss: 0.1059
Epoch 2/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 61s 567ms/step - accuracy: 0.9678 - loss: 0.1098 - val_accuracy: 0.9810 - val_loss: 0.0691
Epoch 3/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 82s 566ms/step - accuracy: 0.9783 - loss: 0.0714 - val_accuracy: 0.9852 - val_loss: 0.0563
Epoch 4/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 59s 546ms/step - accuracy: 0.9826 - loss: 0.0555 - val_accuracy: 0.9862 - val_loss: 0.0539
Epoch 5/5
108/108 ━━━━━━━━━━━━━━━━━━━━ 82s 544ms/step - accuracy: 0.9867 - loss: 0.0449 - val_accuracy: 0.9877 - val_loss: 0.0459


In [25]:
# 테스트 결과 확인
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Test loss:  0.03923730552196503
Test accuracy:  0.9864000082015991
